In [2]:
from utilities.feed import get_stock_info
import utilities.variables as variables 
import yfinance as yf
import pandas as pd
import importlib
import utilities.parser as parser
import utilities.esg_service as esg_service
importlib.reload(parser)
importlib.reload(esg_service)

<module 'utilities.esg_service' from '/Users/herbishtini/Documents/UNI/Master Thesis/sustainability_portfolio_optimisation/utilities/esg_service.py'>

In [6]:
# esg_service.generate_esg_data()
#esg_service.generate_esg_data_for_each_industry()

In [4]:
ticker = yf.Ticker('3462.T')
print(ticker.info)

{'address1': 'Nishishinjuku 8-5-1', 'address2': 'Shinjuku-ku', 'city': 'Tokyo', 'zip': '160-0023', 'country': 'Japan', 'phone': '81 3 3365 8767', 'website': 'https://www.nre-mf.co.jp', 'industry': 'REIT - Retail', 'industryKey': 'reit-retail', 'industryDisp': 'REIT - Retail', 'sector': 'Real Estate', 'sectorKey': 'real-estate', 'sectorDisp': 'Real Estate', 'longBusinessSummary': 'Nomura Real Estate Master Fund, Inc. (NMF) is a real estate investment corporation formed to own and invest primarily in office buildings, retail facilities, logistics and residential facilities. NMF is externally managed by a licensed asset management company, Nomura Real Estate Asset Management Co., Ltd. ("NREAM"). NREAM is a wholly-owned subsidiary of Nomura Real Estate Holdings, Inc. NMF was established on October 1, 2015 through the consolidation type merger involving the former Nomura Real Estate Master Fund, Inc. (hereinafter the "former NMF"), Nomura Real Estate Office Fund, Inc. (hereinafter "NOF") an

In [22]:
# List of stock symbols
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

# Dictionary to store market capitalizations
market_caps = {}

# Fetch market capitalization for each symbol
for symbol in symbols:
    ticker = yf.Ticker(symbol)
    market_cap = ticker.info.get('marketCap', 'pegRatio')  # Use 'N/A' if marketCap is not available
    market_caps[symbol] = market_cap

# Print the market capitalizations
for symbol, market_cap in market_caps.items():
    print(f"{symbol}: {market_cap}")

AttributeError: 'list' object has no attribute 'upper'

In [ ]:
msft.info['marketCap']

In [ ]:
msft.capital_gains

In [ ]:
tickers = yf.Tickers('msft') # 'msft aapl goog'
tickers.tickers['msft'].info

In [ ]:
pd.DataFrame(hist)

In [13]:
#data = yf.download("MSFT", period="1mo") # start="2022-01-01", end=None
data = yf.download("msft", start="1999-01-01", end=None, interval="1mo")
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1999-01-01,34.902344,43.781250,34.000000,43.750000,27.024599,1472518800
1999-02-01,43.859375,43.984375,35.996094,37.531250,23.183243,1599103200
1999-03-01,37.390625,47.312500,36.765625,44.812500,27.680922,1569410000
1999-04-01,45.625000,47.812500,39.937500,40.656250,25.113575,1163599400
1999-05-01,40.718750,40.906250,37.750000,40.343750,24.920549,1124257000
...,...,...,...,...,...,...
2024-03-01,411.269989,430.820007,398.390015,420.720001,419.962494,426349600
2024-04-01,423.950012,429.369995,388.029999,389.329987,388.628998,440777700
2024-05-01,392.609985,433.600006,390.309998,415.130005,414.382568,413800800


In [5]:
# get historical market data
msft = yf.Ticker("MSFT")
hist = msft.history(period="1mo")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-06-20 00:00:00-04:00,446.299988,446.529999,441.269989,445.700012,19877400,0.0,0.0
2024-06-21 00:00:00-04:00,447.380005,450.579987,446.510010,449.779999,34486200,0.0,0.0
2024-06-24 00:00:00-04:00,449.799988,452.750000,446.410004,447.670013,15913700,0.0,0.0
2024-06-25 00:00:00-04:00,448.250000,451.420013,446.750000,450.950012,16747500,0.0,0.0
2024-06-26 00:00:00-04:00,449.000000,453.600006,448.190002,452.160004,16507000,0.0,0.0
2024-06-27 00:00:00-04:00,452.179993,456.170013,451.769989,452.850006,14806300,0.0,0.0
2024-06-28 00:00:00-04:00,453.070007,455.380005,446.410004,446.950012,28362300,0.0,0.0
2024-07-01 00:00:00-04:00,448.660004,457.369995,445.660004,456.730011,17662800,0.0,0.0
2024-07-02 00:00:00-04:00,453.200012,459.589996,453.109985,459.279999,13979800,0.0,0.0
